<a href="https://colab.research.google.com/github/kbehrman/foundational-python-for-data-science/blob/main/Chapter-13%3AFunctional-Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Functional programming and generators

As we have seen, a python program, at it's most basic is composed of a series of statements, simple and compound. The way in which we organize these statements has ramifications for performance, readablility, and ease of modification. Some approaches that have wide adoption are procedural programming, functional programming, and object oriented programming. In this chapter we will introduce some of the concepts of functional programming as well as comprehensions and generators, both of which were borrowed from purley functional langueages. 

### Functional Programming


Functional programming is based on the mathematical definition of functions. A function in this sense, maps an input to an output. For any input there can only be a single output, that is the output for a distinct input will always be the same. There are programming languages which adhere to this limitation strictly, such as Haskall and Erlang. Python is flexible enough that we can adopt some functional concepts, without the strictness. Functional programming in Python is sometimes refered to as Functional Light.

#### State

The state of a program is comprised of names, definintions, and values the exist at a certain time in that program. This in includes function definitions, modules imported, and values assigned to variables. State has whats known as a scope, this is area of the program for which the state holds. Scopes are heirarchical. When we indent a block of code, this code has a nested scope. It inherits scope from the unindented code around it, but does not directly change the outer scope.
In listing 14.1 we set values for the variables a and b in the outer scope. Then in the code block of the function we set a to a different value and print both variables. You can see that when the function is called it uses it's own definition of the variable a, but inherits that for b. In the out scope, the value assigned by the function to a is ignored, as it is out of scope.
 

In [ ]:
a = 'a outer'
b = 'b outer'

def scoped_function():
    a = 'a inner'
    print(a)
    print(b)

scoped_function()
print(a)
print(b)

a inner
b outer
a outer
b outer


### Depending on Global State

So far the code in this book has mostly been presented using the Procedural approach. In this approach the current state is defined by the statements that have run on the lines before the present one. This state is shared through the program, and modified throughout. This means that a function which uses the state to determine it's output could have a different output with the same input. Lets look at some examples contrasting the procedural approach with a functional one.

In listing 14.2 we create a function, describe_the_wind(), which returns a sentence using a variable, wind, defined in the outer scope. You can see that the output of this function will be different depending on this variable. 

In [ ]:
wind = 'Southeast'

def describe_the_wind():
  return f'The wind blows from the {wind}'

describe_the_wind()

'The wind blows from the Southeast'

In [ ]:
wind = 'North'
describe_the_wind()

'The wind blows from the North'

A more functional approach is to pass the variable as a argument, in this way, the function will return the same value for a value passed to it regardless of the outer state: 

In [ ]:
def describe_the_wind(wind):
  return f'The wind blows from the {wind}'

describe_the_wind('Northeast')

'The wind blows from the Northeast'

#### Changing state

In addition to not relying on outside state, a functional function should not directly change outside state. In listing 14.3 we demonastrate a program which changes an outerstate variable, WIND, withing a function change_wind(). Notice the use of the global keyword, this indicates we wish to change an outstate variable rather than defining a new variable in the inner state.

In [ ]:
WINDS = ['Northeast', 'Northwest', 'Southeast', 'Southwest']
WIND = WINDS[0]

def change_wind():
  global WIND
  WIND = WINDS[(WINDS.index(WIND) + 1)%3]
  
WIND

'Northeast'

In [ ]:
change_wind()
WIND

'Northwest'

In [ ]:
for _ in WINDS:
  print(WIND)
  change_wind()


Northwest
Southeast
Northeast
Northwest


A more functional approach to getting the same output is to move the winds variable into the inner state and have the function change_wind() take an argument to determine the output. We demonstrate this in Listing 14.4.

In [ ]:
def change_wind(wind_index):
  winds = ['Northeast', 'Northwest', 'Southeast', 'Southwest']
  return winds[wind_index]

print( change_wind(0) )
print( change_wind(1) )
print( change_wind(2) )
print( change_wind(3) )

Northeast
Northwest
Southeast
Southwest


#### Changing Mutable Data
Another more subtle way of changing outside state is by passing mutable objects. Remember that mutable objects are objects, such as lists and dictionaries, whose contents can be changed. If you set a variable in an outer state, pass it as an argument to a function, and then change it's value in the functions inner state, that the outerstate version of the variable will retain its original value.

In [ ]:
b = 1

def foo(a):
    a = 2

foo(b)

print(b)

1


However, if we pass a mutable object, such as a dictionary as an argument to a function, any change done to that object in the function will be reflected in the outer state as well. Below we define a function which takes a dictionary as an argument and changes one of it's values. You can see that the dictionary, d, when passed to this function, had it's value changed in the outer state:

In [ ]:
d = {"vehicle": "ship", "owner": "Joseph Bruce Ismay"}

def change_mutable_data(data):
  '''A function which changes mutable data.'''
  data['owner'] = 'White Star Line'

  
change_mutable_data(d)
print(d)


{'vehicle': 'ship', 'owner': 'White Star Line'}


Changing mutable objects outside scope in this manner can lead to subtle bugs. One way to avoid this, assuming your data structure isn't too big, is to make a copy in the inner scope, and manipulate this:

In [ ]:
d = {"vehicle": "ship", "owner": "Joseph Bruce Ismay"}

def change_owner(data):
  new_data = data.copy()
  new_data['owner'] = 'White Star Line'
  return new_data


changed = change_owner(d)
changed

{'owner': 'White Star Line', 'vehicle': 'ship'}

By doing this, it is much easier to see where the values are changed. 

### Functional Programming Functions
Three built-in functions that come from the Functional programming world are map(), filter(), and reduce(). 


The map function applys a function to a sequence of values and returns a map object. The input sequence can be any iterable type. That is any object that can be iterated, such as Python Sequences. The map object returned is an iteratable also, so we can loop through it or cast it to a list to view the results:

In [ ]:
def grow_flowers(d):
  return d * "❀"

gardens = map(grow_flowers, [0,1,2,3,4,5])

type(gardens)

map

In [ ]:
list(gardens)

['', '❀', '❀❀', '❀❀❀', '❀❀❀❀', '❀❀❀❀❀']

You can supply map with a function which takes multiple arguments and supply multiple sequences of input values:

In [ ]:
l1 = [0,1,2,3,4]
l2 = [11,10,9,8,7,6]

def multi(d1, d2):
  return d1 * d2

result = map(multi, l1, l2)
print( list(result) )

[0, 10, 18, 24, 28]


Notice that one of the input sequences was longer than the other. The map function will stop when it reaches the end of the shortest input sequence.

The reduce() function also takes a function and an iterable as arguments. It then uses the function to return a single value based on the input. For example, if we want to subtract debit from an account balance, we could do it with a for loop like this: 

In [ ]:
initial_balance = 10000
debits = [20, 40, 300, 3000, 1, 234]

balance = initial_balance

for debit in debits:
  balance -= debit
  
balance

6405

Or we could achieve the same result using the reduce() function like this:

In [ ]:
from functools import reduce

inital_balance = 10000
debits = [20, 40, 300, 3000, 1, 234]

def minus(a, b):
  return a - b

balance = reduce(minus, debits, initial_balance)
balance

6405

The operator module provides all of the standard operators as functions, including functions for the standard mathimatical operations. We can use the operator.sub() function as an argument to reduce() as a replacement for our minus() function:

In [ ]:
from functools import reduce
import operator

inital_balance = 10000
debits = [20, 40, 300, 3000, 1, 234]

reduce(operator.sub, debits, initial_balance) 

6405

The filter() function takes a function and an iterable as arguments. The function should return True or False based on each item. The result is a iterable object of only input values that caused the function to return True. For example, to get only the capital letters from a string, we defin a function that tests whether a character is capitalized, and pass it and the string to filter():

In [ ]:
charles = 'ChArlesTheBald'

def is_cap(a):
  return a.isupper()

retval = filter(is_cap, charles)

list(retval)

['C', 'A', 'T', 'B']

Theses three methods are one of the few times we really recomend using lambda functions. You do doing a simple comparison, say for all the numbers less than 10 and greater than 3, by a lambda function and range() as arguments in a clean and easy to read way:

In [ ]:
nums = filter(lambda x: x > 3, range(10))
list(nums)

[4, 5, 6, 7, 8, 9]

#### List Comprehensions
List comprehensions are syntax borred from the functional language Haskell (https://docs.python.org/3/howto/functional.html). You can think of them as a one-line for loop which returns a list. Though there source is in functional programming, their use has become standard in all Python approches. 

The basic syntax for a list comprehension is :

[ \<item returned\> for \<source item\> in \<iterable\> ] 

For example, given a list of names we wish to change to title capitalization, we would use x.title() as the item returned and each name as a source item:

In [ ]:
names = ['tim', 'tiger', 'tabassum', 'theodora', 'tanya']

capd = [x.title() for x in names]

capd

['Tim', 'Tiger', 'Tabassum', 'Theodora', 'Tanya']

The equivalent process using a for loop would be:

In [ ]:
names = ['tim', 'tiger', 'tabassum', 'theodora', 'tanya']

capd = []

for name in names:
  capd.append(name.title())
  
capd

['Tim', 'Tiger', 'Tabassum', 'Theodora', 'Tanya']

List comprehensions can be used as replacements for the map() and filter() functions. We can replace this code, which maps the number 0-5 with a function which inserts them into a string:

In [ ]:
def count_flower_petals(d):
  return f"{d} petals counted so far"

counts = map(count_flower_petals, range(6))

list(counts)

['0 petals counted so far',
 '1 petals counted so far',
 '2 petals counted so far',
 '3 petals counted so far',
 '4 petals counted so far',
 '5 petals counted so far']

With this much simplier list comprehension:

In [ ]:
[f"{x} petals counted so far" for x in range(6)]

['0 petals counted so far',
 '1 petals counted so far',
 '2 petals counted so far',
 '3 petals counted so far',
 '4 petals counted so far',
 '5 petals counted so far']

We can replace our filter() example, returning only capital letters:

In [ ]:
characters = ['C', 'b', 'c', 'A', 'b','P', 'g', 'S']

def cap(a):
  return a.isupper()

retval = filter(cap, characters)

list(retval)

['C', 'A', 'P', 'S']

With this list comprehension using a conditional:

In [ ]:
characters = ['C', 'b', 'c', 'A', 'b','P', 'g', 'S']

[x for x in characters if x.isupper()]

['C', 'A', 'P', 'S']

The syntax for using a conditional with a list comprehension is:

[ \<item returned\> for \<source item\> in \<iterable\> if \<condition\> ]

It the items in the source iterable are sequences, we can unpack them by using multiple variables:

In [ ]:
points = [(12, 3), (-1, 33), (12, 0)]

[ f'x: {x} y: {y}' for x, y in points ]

['x: 12 y: 3', 'x: -1 y: 33', 'x: 12 y: 0']


We can perform the equivalant of nested for loops by using multiple for..in statements in the same list comprehensions:

In [ ]:
list_of_lists = [[1,2,3], [4,5,6], [7,8,9]]


[x for y in list_of_lists for x in y]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

#### Dict comprehensions

Dictionary comprehensions rely on a similar syntax, but instead of having a single value appended to a list, a key value pair is added to a dictionary. Here is an example where we use the values in two lists to construct a dictionary.

In [ ]:
names = ['James', 'Jokubus', 'Shaemus']
scores = [12, 33, 23]

{ name:score for name in names for score in scores}


{'James': 23, 'Jokubus': 23, 'Shaemus': 23}

### Generators

One of the big advantages of a range object over a list when dealing with big numerical ranges is that the range object calculates results as you request them. This means that it's memory footprint is consitently small. Generators let us use our own calculations to create values on demand, in a similar way as range objects.

#### Generator Expressions

One way to create generators is through generator expressions. These use the same syntax as list comprehensions except that the enclosing square brackets are replaced with parenthesis. In the next example we create a list and a generator based on the same calculation and print them.

In [ ]:
l_ten = [x**3 for x in range(10)]
g_ten = (x**3 for x in range(10))

print(f"l_ten is a {type(l_ten)}")
print(f"l_ten prints as: {l_ten}")
print(f"g_ten is a {type(g_ten)}")
print(f"g_ten prints as: {g_ten}")

l_ten is a <class 'list'>
l_ten prints as: [0, 1, 8, 27, 64, 125, 216, 343, 512, 729]
g_ten is a <class 'generator'>
g_ten prints as: <generator object <genexpr> at 0x7f29bfb3c850>


When we print the list, we can see it's contents, not so with a generator. To get a value from a generator we have to request the next one, we can do this using the next() function:

In [ ]:
next(g_ten)

0

Or more commonly, by using iterating through it using a loop:

In [ ]:
for x in g_ten:
  print(x)


1
8
27
64
125
216
343
512
729


Because generator only generate values on demand, there is no way to index or slice them:

In [ ]:
g_ten[3]

TypeError: ignored

One of the important advantages of generators over lists is there memory footprint. Here we use the sys.getsizeof() function to compare the sizes of a list and a generator:

In [ ]:
import sys
x = 100000000
l_big = [x for x in range(x)]
g_big = (x for x in range(x))

print( f"l_big is {sys.getsizeof(l_big)} bytes")
print( f"g_big is {sys.getsizeof(g_big)} bytes")

l_big is 859724480 bytes
g_big is 128 bytes


##### Generator functions

You can use generator functions to create generators that are more complex. These look like normal functions with the return statement replaced with a yeild statement. The generator will keep it's own internal state, returning values as requested:

In [ ]:
def square_them(numbers):
  for number in numbers:
    yield number * number
    
  
s = square_them(range(10000))

print(next(s))
print(next(s))
print(next(s))
print(next(s))

0
1
4
9



An additional advantage of generators over lists is the ability to create an infinite generator. This is a generator with no end! It will return as many values as requested. For example, we can make a generator which increments a number as many times as we like:

In [ ]:
def counter(d):
  
  while True:
    d += 1
    yield d

In [ ]:
c = counter(10)

print(next(c))
print(next(c))
print(next(c))

11
12
13


In listing... we chain four generators together. This is a useful way to keep each generator understandable, while still harnessing their just in time calculations.

In [ ]:
evens = (x*2 for x in range(5000000))
three_factors = (x//3 for x in evens if x%3 == 0)
titles = (f"this number is {x}" for x in three_factors)
capped = (x.title() for x in titles)

print(f"The first call to capped: {next(capped)}")
print(f"The second call to capped: {next(capped)}")
print(f"The third call to capped: {next(capped)}")

The first call to capped: This Number Is 0
The second call to capped: This Number Is 2
The third call to capped: This Number Is 4


Generators are a great way to make your code performant. You should consider using them when ever you are iterating through a long sequence of calculated values.

## Summary
Functional programming is a approach to organizing your programs which is useful for designing software which can be run concurrently. It is based on the idea that a functions inner state should be changed by or change the outer state of the code calling it. A function should always return the same value for a given input. Some built-in functions that come from the functional programming world are map(), filter(), and reduce(). List comprehensions and generators are both very Pythonic ways of creating a sequence of values. Generators are recomended when iteratoring through any large number of values, or when you don't know how many values you need.
